**Import necessary package**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.misc import imread
from IPython import display
from PIL import Image
%matplotlib inline

In [3]:
import os
import cv2
from tqdm import tqdm

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

**Data Pre-Processing**

In [ ]:
REBUILD_DATA = True

class EISType():
    IMG_SIZE = 32
    
    # A = ""  Determine the number of type and then give the directory of each type of image
    # B = ""
    # C = ""
    # D = ""
    # E = ""
    LABELS = {A:0, B:1, C:2, D:3, E:4}
    training_data = []
    acount = 0
    bcount = 0
    ccount = 0
    dcount = 0
    ecount = 0
    


**NN Model**

In [5]:
class Net(nn.Module):
    
    def __init__(self, input_size, hidden_size, kernel_size, output_size):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(input_size, hidden_size, kernel_size) 
        self.conv2 = nn.Conv2d(hidden_size, hidden_size*2, kernel_size)
        self.conv3 = nn.Conv2d(hidden_size*2, hidden_size*4, kernel_size)
        self.conv4 = nn.Conv2d(hidden_size*4, hidden_size*8, kernel_size)
        
        self.fc1 = nn.Linear(68610, 64) 
        self.fc2 = nn.Linear(64, output_size)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (3,3))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv4(x)), (2,2))
        
        xF = x.view(-1, 64 * 71 * 15 ) # flatten
        output = F.relu(self.fc1(xF)) # put into the first fully connected layer
        output = self.fc2(x)
        
        return F.softmax(output, dim=1)

In [6]:
net = Net(1, 8, 25, 5)